In [ ]:
from qiskit import *
from qiskit.visualization import plot_histogram

In [ ]:
# Making the Qiskit Classes:
qreg_s1_to_m2 = QuantumRegister(1, 's1_to_m2')
qreg_s2_to_m1 = QuantumRegister(1, 's2_to_m1')
qreg_to_m3 = QuantumRegister(1, 'to_m3')
creg_measurements = ClassicalRegister(3, 'measurements')
circuit = QuantumCircuit(
    qreg_s1_to_m2,
    qreg_s2_to_m1,
    qreg_to_m3,
    creg_measurements
    )

# Initializing the Circuit:
ket_0 = [1, 0]
ket_1 = [0, 1]
quantumRead = [ket_1, ket_0]   # |10>

circuit.initialize(quantumRead[0], [qreg_s1_to_m2[0]])
circuit.initialize(quantumRead[1], [qreg_s2_to_m1[0]])

# Designing the circuit
circuit.ccx(qreg_s1_to_m2[0], qreg_s2_to_m1[0], qreg_to_m3[0])
circuit.x(qreg_s1_to_m2[0])
circuit.x(qreg_s2_to_m1[0])
circuit.barrier()
circuit.measure(qreg_to_m3[0], creg_measurements[0])
circuit.measure(qreg_s1_to_m2[0], creg_measurements[1])
circuit.measure(qreg_s2_to_m1[0], creg_measurements[2])
circuit.draw(output = 'mpl')

In [ ]:
comp = Aer.get_backend("qasm_simulator")
results = execute(circuit, comp).result()
# plot_histogram(results.get_counts(circuit))
list(results.get_counts(circuit).keys())

In [ ]:
from qiskit.providers.ibmq import least_busy
from qiskit.tools.monitor import job_monitor
from qiskit.providers.ibmq.ibmqbackend import IBMQSimulator
IBMQ.load_account()

In [ ]:
provider = IBMQ.get_provider('ibm-q')
quantum_backends = [x for x in provider.backends() if type(x) != IBMQSimulator]
q_comp = least_busy(quantum_backends)
print(f"running on '{q_comp.name()}'")

In [ ]:
job = execute(circuit, q_comp, shots = 16384)
job_monitor(job)
print("results:", job.result().get_counts(circuit))
plot_histogram(job.result().get_counts(circuit))